# Flower Recognition Project 
*PyTorch & Computer Vision for AI Engineering Master with ProfessionAI*

## Google Colab configuration

If you're testing this project on Google Colab, it could be useful for you to run the following cells

In [ ]:
# If you are running this code on Google Colab, run this:

!git clone https://github.com/Silvano315/PyTorch-CNN-for-food-image-classification-system.git

%cd PyTorch-CNN-for-food-image-classification-system
%pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/Project_PyTorch_ProfessionAI/dataset/ /content/PyTorch-CNN-for-food-image-classification-system/

## Import Libraries and initial Set-Up

In [1]:
# Import libraries

import os
import sys
import json
import random
import logging
import time

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.constants import RANDOM_SEED, DATA_PATH, BATCH_SIZE
from src.utils import extract_dataset, get_paths_to_files, get_dataset_paths, get_logger
from src.visualization import display_random_images, visualize_class_samples, plot_class_distribution, compare_class_distribution, \
                        analyze_image_dimensions, analyze_color_distribution, visualize_random_images, visualize_augmented_images, \
                        scatter_plot_metrics, plot_confusion_matrix 
from src.preprocessing import create_datasets, create_data_loaders
from src.models import create_model, Experiment, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, train_model, validate, \
                        get_predictions, freeze_layers

In [2]:
# Set Random Seed for reproducibility

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
# Check if GPU is available and set it as device 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This repository is connected to {str(device).upper()}")

This repository is connected to CPU


## Extract zipped Dataset

In [4]:
# Extract Dataset and save files to dataset folder

extract_dataset('progetto-finale-flowes.tar.gz', 'dataset/')

Extracting files: 100%|██████████| 3674/3674 [00:00<00:00, 5980.24file/s]


Dataset extracted to dataset
Total files extracted: 3674
Removing macOS hidden files...
Removed 1841 macOS hidden files/folders
Final number of files: 1833


## Exploratory Data Analysis

In [5]:
# Get paths and filenames form directory dataset 

filepaths, filenames = get_paths_to_files(DATA_PATH)

print("="*80)
print("File paths:")
for i in range(0,5):
    print(filepaths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(filenames[i])
print("="*80)

File paths:
dataset/progetto-finale-flowes/train/daisy/1879567877_8ed2a5faa7_n_jpg.rf.38fa4a22817b8f3b37f50d22ea608f96.jpg
dataset/progetto-finale-flowes/train/daisy/7702332000_3f21ef4571_n_jpg.rf.938ac8ed1d43f09d9566cdd327339ddc.jpg
dataset/progetto-finale-flowes/train/daisy/2454280137_e1637536ae_n_jpg.rf.f7b21fb60a95a578b5c4f1a880fa1887.jpg
dataset/progetto-finale-flowes/train/daisy/12701063955_4840594ea6_n_jpg.rf.058be03c02d95624d8c201fe3eedb333.jpg
dataset/progetto-finale-flowes/train/daisy/7630517248_98fb8bee1f_n_jpg.rf.3fa9ef879de7039bf2c346681d49e6e8.jpg
File names:
1879567877_8ed2a5faa7_n_jpg.rf.38fa4a22817b8f3b37f50d22ea608f96.jpg
7702332000_3f21ef4571_n_jpg.rf.938ac8ed1d43f09d9566cdd327339ddc.jpg
2454280137_e1637536ae_n_jpg.rf.f7b21fb60a95a578b5c4f1a880fa1887.jpg
12701063955_4840594ea6_n_jpg.rf.058be03c02d95624d8c201fe3eedb333.jpg
7630517248_98fb8bee1f_n_jpg.rf.3fa9ef879de7039bf2c346681d49e6e8.jpg


In [6]:
# Access to path dirs and file names for each split (test, train, val)

dataset_paths = get_dataset_paths(DATA_PATH)

train_paths, train_names = dataset_paths['train']
test_paths, test_names = dataset_paths['test']
val_paths, val_names = dataset_paths['valid']

print("="*80)
print("File paths:")
for i in range(0,5):
    print(train_paths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(train_names[i])
print("="*80)

File paths:
dataset/progetto-finale-flowes/train/daisy/1879567877_8ed2a5faa7_n_jpg.rf.38fa4a22817b8f3b37f50d22ea608f96.jpg
dataset/progetto-finale-flowes/train/daisy/7702332000_3f21ef4571_n_jpg.rf.938ac8ed1d43f09d9566cdd327339ddc.jpg
dataset/progetto-finale-flowes/train/daisy/2454280137_e1637536ae_n_jpg.rf.f7b21fb60a95a578b5c4f1a880fa1887.jpg
dataset/progetto-finale-flowes/train/daisy/12701063955_4840594ea6_n_jpg.rf.058be03c02d95624d8c201fe3eedb333.jpg
dataset/progetto-finale-flowes/train/daisy/7630517248_98fb8bee1f_n_jpg.rf.3fa9ef879de7039bf2c346681d49e6e8.jpg
File names:
1879567877_8ed2a5faa7_n_jpg.rf.38fa4a22817b8f3b37f50d22ea608f96.jpg
7702332000_3f21ef4571_n_jpg.rf.938ac8ed1d43f09d9566cdd327339ddc.jpg
2454280137_e1637536ae_n_jpg.rf.f7b21fb60a95a578b5c4f1a880fa1887.jpg
12701063955_4840594ea6_n_jpg.rf.058be03c02d95624d8c201fe3eedb333.jpg
7630517248_98fb8bee1f_n_jpg.rf.3fa9ef879de7039bf2c346681d49e6e8.jpg


In [ ]:
# Visualize n random images 

fig, axes = display_random_images(filepaths, n=25)
plt.show() 

In [ ]:
# Visualize n random images from a chosen split dataset (train_paths, test_paths, val_paths)

chosen_split = train_paths

fig, axes = display_random_images(chosen_split, n=25)
plt.show() 

In [ ]:
# Displays sample images for each class in the chosen split set

chosen_split = train_paths

fig = visualize_class_samples(chosen_split, num_samples=3, max_classes=2)
plt.show()

In [ ]:
# Bar plot for class distribution for each dataset split

chosen_split = train_paths

class_dist_fig = plot_class_distribution(chosen_split)
class_dist_fig.show()

In [ ]:
# Comparison of the Class Distribution between Train, Test e Validation

comparison_fig = compare_class_distribution(dataset_paths)
comparison_fig.show()

In [ ]:
# Several plots to analyze Images Dimensions

dim_fig = analyze_image_dimensions(val_paths)
dim_fig.show()

In [ ]:
# Histogram Plots to analyze Colour Distribution for a chosen split dataset

chosen_split = train_paths

color_dist_fig = analyze_color_distribution(chosen_split, n_samples=50)
color_dist_fig.show()

## Preprocessing with image visualization

In [12]:
# Preprocessing for Baseline Model (without augmentation, also for train set)
# Resize Dimensions (224,224) and Normalization

trainset, valset, testset = create_datasets(DATA_PATH, augment_train=False)

print('='*60 + "\nTrain:")
print(trainset)
print('='*60 + "\Val:")
print(valset)
print('='*60 + "\Test:")
print(testset)
print("="*60)

Train:
Dataset ImageFolder
    Number of datapoints: 1275
    Root location: dataset/progetto-finale-flowes//train
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a05c3d90>
============================================================\Val:
Dataset ImageFolder
    Number of datapoints: 364
    Root location: dataset/progetto-finale-flowes//valid
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a0d37bd0>
============================================================\Test:
Dataset ImageFolder
    Number of datapoints: 182
    Root location: dataset/progetto-finale-flowes//test
    StandardTransform
Transform: <src.preprocessing.Transforms object at 0x2a0d37bd0>


In [ ]:
# Visualization of random images from ImageFolder

chosen_folder = trainset

visualize_random_images(chosen_folder, num_images=16, axis=False)

In [ ]:
# Preprocessing for Transfer Learning Model (with augmentation for train set)
# Resize Dimensions (224,224) and Normalization

trainset_aug, valset, testset = create_datasets(DATA_PATH, augment_train=True)

print('='*60 + "\nTrain:")
print(trainset_aug)
print('='*60 + "\Val:")
print(valset)
print('='*60 + "\Test:")
print(testset)
print("="*60)

In [ ]:
# Visualization of random augmented images from ImageFolder

visualize_augmented_images(trainset_aug, num_images=5)

In [ ]:
# Create DataLoader for each dataset split

is_augmented = False
datasets = {
    'train': trainset_aug if is_augmented else trainset,
    'val': valset,
    'test': testset
}

dataloaders = create_data_loaders(datasets, batch_size=BATCH_SIZE, num_workers=0)

train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

## CNN model: transfer learning with timm

### Preprocessing

In [ ]:
# Preprocessing for Transfer Learning Model (with augmentation for train set)
# Resize Dimensions (224,224) and Normalization

trainset_aug, valset, testset = create_datasets(DATA_PATH, augment_train=True)

print('='*60 + "\nTrain:")
print(trainset_aug)
print('='*60 + "\Val:")
print(valset)
print('='*60 + "\Test:")
print(testset)
print("="*60)

In [ ]:
# Create DataLoader for each dataset split

is_augmented = True
datasets = {
    'train': trainset_aug if is_augmented else trainset,
    'val': valset,
    'test': testset
}

dataloaders = create_data_loaders(datasets, batch_size=BATCH_SIZE, num_workers=0)

train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

### Transfer Learning model creation and training

In [ ]:
# Define configurations

num_epochs = 100
learning_rate = 0.001
num_classes = len(trainset_aug.classes)

In [ ]:
# Create model and  set to device

logger = get_logger(ch_log_level=logging.INFO, fh_log_level=logging.DEBUG)

model = create_model(num_classes, model_type='efficientnet', efficientnet_version='b0')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)